# Big-means: K-means for big data clustering

Rustam Mussabayev, Nenad Mladenovic, Bassem Jarboui, Ravil Mussabayev. How to Use K-means for Big Data Clustering? // Pattern Recognition, Volume 137, May 2023, 109269; 
https://doi.org/10.1016/j.patcog.2022.109269

In [1]:
from bigmeans import *
import math
import csv
import numpy as np
import matplotlib.pyplot as plt

def load_dataset():
    filename = 'skin_segmentation.data'
    columns = slice(0, 3)
    with open(filename, newline='') as f1:
        reader = csv.reader(f1, delimiter='\t')
        raw = [row[columns] for row in reader]
    return np.array(raw, dtype=float)

points = load_dataset()


# Big-means parameters:
sample_size = 160000 # The number of data points to be randomly selected from the input dataset at each iteration of the Big-means.
n_centers = 25 # The desired number of clusters
max_iter = 2000 # Maximum number of samples to be processed
tmax = 1000.0 # The time limit for the search process (in seconds); a zero or negative value means no limit.
local_max_iters = 300 # The maximum number of K-means iterations before declaring convergence and stopping the clustering process for each sample.
local_tol = 0.0001 # The threshold below which the relative change in the objective function between two iterations must fall to declare convergence of K-means.
n_candidates = 3 # The number of candidate centers to choose from at each stage of the K-means++ initialization algorithm



nb.set_num_threads(nb.config.NUMBA_NUM_THREADS) # Set the number of threads for parallel execution to the maximum possible
#nb.set_num_threads(3) # Set the number of threads for parallel execution to the some value

# Best Known Solution (for comparison)
f_best = 102280000


In [3]:
%%time
print('SEQUENTIAL BIG-MEANS:')
print()
centers, objective, assignment, n_iter, best_n_iter, best_time, n_dists = big_means_sequential(points, n_centers, sample_size, max_iter, tmax, local_max_iters, local_tol, n_candidates, True)
print()
print('#Iterations: ', n_iter)
print('#Distances: ', n_dists)
print('Full Objective: ', objective)
objective_gap = round((objective - f_best) / objective * 100, 2)
print('Objective Gap: ', objective_gap, '%')
print()

SEQUENTIAL BIG-MEANS:

sample objective              n_iter         cpu_time       
69293741.229442               1              0.43           
69223473.846065               5              0.64           
69022076.218531               6              0.68           
68964761.587398               7              0.73           
68834556.997361               11             0.93           
68511068.952671               38             2.28           
68495261.735416               908            46.29          
68487201.386202               1929           98.66          

#Iterations:  2000
#Distances:  24981806425
Full Objective:  106035132.15935105
Objective Gap:  3.54 %

CPU times: user 1min 42s, sys: 30 ms, total: 1min 42s
Wall time: 1min 42s


In [5]:
%%time
print("BIG-MEANS WITH 'INNER PARALLELISM':")
print()
centers, objective, assignment, n_iter, best_n_iter, best_time, n_dists = big_means_inner(points, n_centers, sample_size, max_iter, tmax, local_max_iters, local_tol, n_candidates, True)
print()
print('#Iterations: ', n_iter)
print('#Distances: ', n_dists)
print('Full Objective: ', objective)
objective_gap = round((objective - f_best) / objective * 100, 2)
print('Objective Gap: ', objective_gap, '%')
print()

BIG-MEANS WITH 'INNER PARALLELISM':

sample objective              n_iter         cpu_time       
70074694.464533               1              0.13           
70019288.972940               6              0.20           
69962827.253794               9              0.27           
69717493.239673               19             0.44           
69625523.429908               34             0.68           
69605115.011869               368            6.02           
69552421.627137               383            6.34           
69500734.236141               406            6.70           
69426186.951997               701            11.39          
69425156.067758               1014           16.30          
69247934.559932               1604           26.03          

#Iterations:  2000
#Distances:  24273806425
Full Objective:  107626144.49978782
Objective Gap:  4.97 %

CPU times: user 5min 12s, sys: 734 ms, total: 5min 13s
Wall time: 32.6 s


In [8]:
%%time
print("BIG-MEANS WITH 'COMPETITIVE PARALLELISM':")
print()
centers, objective, assignment, n_iter, best_n_iter, best_time, n_dists = big_means_competitive(points, n_centers, sample_size, max_iter, tmax, local_max_iters, local_tol, n_candidates, True)
print()
print('#Iterations: ', n_iter)
print('#Distances: ', n_dists)
print('Full Objective: ', objective)
objective_gap = round((objective - f_best) / objective * 100, 2)
print('Objective Gap: ', objective_gap, '%')
print()

BIG-MEANS WITH 'COMPETITIVE PARALLELISM':

sample objective              n_iter         cpu_time       
71010453.196253               1              1.14           
69590621.491709               2              1.16           
70270655.933662               3              1.17           
70726779.523461               4              1.18           
69335159.995975               5              1.23           
74039588.963294               6              1.25           
69363697.432248               9              1.30           
70037853.587414               13             1.40           
69440090.561419               16             1.42           
70133179.934012               19             1.48           
69832767.857292               24             1.52           
67626233.097587               25             1.52           
69224169.917026               27             1.55           
69237519.036051               32             1.62           
67250324.420984               37          

In [9]:
%%time
print("BIG-MEANS WITH 'COLLECTIVE PARALLELISM':")
print()
centers, objective, assignment, n_iter, best_n_iter, best_time, n_dists = big_means_collective(points, n_centers, sample_size, max_iter, tmax, local_max_iters, local_tol, n_candidates, True)
print()
print('#Iterations: ', n_iter)
print('#Distances: ', n_dists)
print('Full Objective: ', objective)
objective_gap = round((objective - f_best) / objective * 100, 2)
print('Objective Gap: ', objective_gap, '%')
print()

BIG-MEANS WITH 'COLLECTIVE PARALLELISM':

sample objective              n_iter         cpu_time       
71134396.476587               1              1.23           
71344463.845906               2              1.24           
72835056.080045               3              1.27           
70194712.761034               4              1.30           
68890254.335119               5              1.30           
70871325.010844               6              1.35           
70854206.972676               7              1.36           
70593604.704333               8              1.39           
71426082.708606               9              1.40           
72762996.489543               12             1.44           
68629365.794029               13             1.46           
70453883.021562               14             1.47           
68808551.911702               18             1.54           
72682458.115644               20             1.55           
69171808.973895               29           

In [9]:
%%time
print("BIG-MEANS WITH 'HYBRID PARALLELISM':")
print()
centers, objective, assignment, n_iter, best_n_iter, best_time, n_dists = big_means_hybrid(points, n_centers, sample_size, max_iter, max_iter, tmax, tmax, local_max_iters, local_tol, n_candidates, True)
print()
print('#Iterations: ', n_iter)
print('#Distances: ', n_dists)
print('Full Objective: ', objective)
objective_gap = round((objective - f_best) / objective * 100, 2)
print('Objective Gap: ', objective_gap, '%')
print()

BIG-MEANS WITH 'HYBRID PARALLELISM':

sample objective              n_iter         cpu_time       
76905659.478232               1              1.15           
68126041.510885               3              1.27           
72307454.695430               4              1.28           
68920421.121746               5              1.30           67708695.192375               6              1.30           

68374544.072110               7              1.33           
71481797.351898               8              1.35           
67472299.790755               12             1.42           
71627396.122817               16             1.47           
70490738.468369               20             1.52           
67389620.297408               21             1.53           
67140187.769132               29             1.64           
71528225.744739               37             1.75           
69970263.509243               41             1.80           
69099336.328487               50             1.